<a href="https://colab.research.google.com/github/Alex31y/MortadellAI-hackatlon/blob/main/MortadellAI_1_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**changelog v1.2**:


*   Aggiunta la possibilità di far selezionare all'utente il livello di competenza a proposito del documento dato in input
*   Ad un errore nella risposta del quiz, viene generata una *lezione personalizzata*
*   Raffinamento del prompt per la generazione del corpus:
>ora è più prolisso ma può essere ripetitivo / poco scorrevole
*   Generazione text to speech di un file .wav del podcast
>il modello per la generazione viene fatto girare in "locale", rallentando significativamente i tempo di computazione dell'applicazione




**Librerie da installare**

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user
!pip install gradio PyPDF2
!pip install git+https://github.com/suno-ai/bark.git

**Blocco per il riavvio del runtime, da eseguire solo una volta dopo l'installazione delle librerie sopra**

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Autenticazione**

per far girare l'applicazione in locale seguire la seguente guida:

https://cloud.google.com/docs/authentication/provide-credentials-adc?hl=it#how-to

In [1]:
import gradio as gr
import PyPDF2
import vertexai
from vertexai.preview.language_models import TextGenerationModel
from google.colab import auth
auth.authenticate_user()

**Configurazione del modello**

**LLM DI GOOGLE**

Sintetizzazione del testo

In [2]:
def riassumi(articolo):
  vertexai.init(project="gen-mortadellai", location="us-central1")
  parameters = {
      "temperature": 0.2,
      "max_output_tokens": 1024,
      "top_p": 0.95,
      "top_k": 40
  }
  model = TextGenerationModel.from_pretrained("text-bison@001")
  response = model.predict(
      f"""Provide a summary for the following article: Beyond our own products, we think it\'s important to make it easy, safe and scalable for others to benefit from these advances by building on top of our best models. Next month, we\'ll start onboarding individual developers, creators and enterprises so they can try our Generative Language API, initially powered by LaMDA with a range of models to follow. Over time, we intend to create a suite of tools and APIs that will make it easy for others to build more innovative applications with AI. Having the necessary compute power to build reliable and trustworthy AI systems is also crucial to startups, and we are excited to help scale these efforts through our Google Cloud partnerships with Cohere, C3.ai and Anthropic, which was just announced last week. Stay tuned for more developer details soon.
  Summary: Google is making its AI technology more accessible to developers, creators, and enterprises. Next month, Google will start onboarding developers to try its Generative Language API, which will initially be powered by LaMDA. Over time, Google intends to create a suite of tools and APIs that will make it easy for others to build more innovative applications with AI. Google is also excited to help scale these efforts through its Google Cloud partnerships with Cohere, C3.ai, and Anthropic.

  Provide a summary with about two sentences for the following article: The benefits of electricPromptData kitchens go beyond climate impact, starting with speed. The first time I ever cooked on induction (electric) equipment, the biggest surprise was just how incredibly fast it is. In fact, induction boils water twice as fast as traditional gas equipment and is far more efficient — because unlike a flame, electric heat has nowhere to escape. At Bay View, our training programs help Google chefs appreciate and adjust to the new pace of induction. The speed truly opens up whole new ways of cooking.
  Summary: Electric kitchens are faster, more efficient, and better for the environment than gas kitchens. Induction cooking is particularly fast, boiling water twice as fast as traditional gas equipment. This speed opens up whole new ways of cooking. Google chefs are trained to appreciate and adjust to the new pace of induction cooking at Bay View.

  Provide a summary with about two sentences for the following article: We\'re also using AI to forecast floods, another extreme weather pattern exacerbated by climate change. We\'ve already helped communities to predict when floods will hit and how deep the waters will get — in 2021, we sent 115 million flood alert notifications to 23 million people over Google Search and Maps, helping save countless lives. Today, we\'re sharing that we\'re now expanding our coverage to more countries in South America (Brazil and Colombia), Sub-Saharan Africa (Burkina Faso, Cameroon, Chad, Democratic Republic of Congo, Ivory Coast, Ghana, Guinea, Malawi, Nigeria, Sierra Leone, Angola, South Sudan, Namibia, Liberia, and South Africa), and South Asia (Sri Lanka). We\'ve used an AI technique called transfer learning to make it work in areas where there\'s less data available. We\'re also announcing the global launch of Google FloodHub, a new platform that displays when and where floods may occur. We\'ll also be bringing this information to Google Search and Maps in the future to help more people to reach safety in flooding situations.
  Summary: Google is using AI to forecast floods in South America, Sub-Saharan Africa, South Asia, and other parts of the world. The AI technique of transfer learning is being used to make it work in areas where there\'s less data available. Google FloodHub, a new platform that displays when and where floods may occur, has also been launched globally. This information will also be brought to Google Search and Maps in the future to help more people reach safety in flooding situations.

  Provide a summary with about two sentences for the following article: In order to learn skiing, you must first be educated on the proper use of the equipment. This includes learning how to properly fit your boot on your foot, understand the different functions of the ski, and bring gloves, goggles etc. Your instructor starts you with one-footed ski drills. Stepping side-to-side, forward-and-backward, making snow angels while keeping your ski flat to the ground, and gliding with the foot not attached to a ski up for several seconds. Then you can put on both skis and get used to doing them with two skis on at once. Next, before going down the hill, you must first learn how to walk on the flat ground and up small hills through two methods, known as side stepping and herringbone. Now it\'s time to get skiing! For your first attempted run, you will use the skills you just learned on walking up the hill, to go down a small five foot vertical straight run, in which you will naturally stop on the flat ground. This makes you learn the proper athletic stance to balance and get you used to going down the hill in a safe, controlled setting. What do you need next? To be able to stop yourself. Here, your coach will teach you how to turn your skis into a wedge, also commonly referred to as a pizza, by rotating legs inward and pushing out on the heels. Once learned, you practice a gliding wedge down a small hill where you gradually come to a stop on the flat ground thanks to your wedge. Finally, you learn the necessary skill of getting up after falling, which is much easier than it looks, but once learned, a piece of cake.
  Summary: Skiing is a great way to enjoy the outdoors and get some exercise. It can be a little daunting at first, but with a little practice, you\'ll be skiing like a pro in no time.

  Provide a summary with about two sentences for the following article: Yellowstone National Park is an American national park located in the western United States, largely in the northwest corner of Wyoming and extending into Montana and Idaho. It was established by the 42nd U.S. Congress with the Yellowstone National Park Protection Act and signed into law by President Ulysses S. Grant on March 1, 1872. Yellowstone was the first national park in the U.S. and is also widely held to be the first national park in the world.The park is known for its wildlife and its many geothermal features, especially the Old Faithful geyser, one of its most popular. While it represents many types of biomes, the subalpine forest is the most abundant. It is part of the South Central Rockies forests ecoregion.
  Summary: Yellowstone National Park is the first national park in the United States and the world. It is located in the western United States, largely in the northwest corner of Wyoming and extending into Montana and Idaho. The park is known for its wildlife and its many geothermal features, especially the Old Faithful geyser.

  Provide a summary for the following article: {articolo}
  Summary:
  """,
      **parameters
  )
  return response.text

Introduzione del podcast

In [26]:
def romanza(context, expertise):
    print(f"\n\nLivello di expertise: {expertise}\n")
    vertexai.init(project="gen-mortadellai", location="us-central1")
    parameters = {
        "temperature": 0.2,
        "max_output_tokens": 1024,
        "top_p": 0.95,
        "top_k": 40
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        f"""Imagine you are the host of a popular podcast called Il postcast di mortadellAI. Your show is known for its captivating storytelling and immersive atmosphere. You are about to dive into a new episode, and you want to engage your audience from the very beginning. The level of expertise of your audience is: {expertise}. You have some context that you would like to share with your listeners. Start with an introduction that entices and hooks your audience, setting the stage for an unforgettable podcast experience. Then continue the story covering all of the input context as if they were {expertise} of the topic. Make sure to incorporate vivid descriptions, evocative language, and a sense of anticipation that will leave your listeners eagerly waiting for what\'s to come. Remember, your goal is to create an atmosphere of intrigue and captivate your audience\'s imagination. Finally end with a grand finale
    context:
    {context}
    """,
        **parameters
    )
    return response.text

Corpo del podcast

In [4]:
def corpo(facts):
    vertexai.init(project="gen-mortadellai", location="us-central1")
    parameters = {
        "temperature": 0.2,
        "max_output_tokens": 256,
        "top_p": 0.8,
        "top_k": 40
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        f"""don\'t be repetitive

    Summarize the following facts in an engaging format: [input facts]
    units and 6,480 people – representing approximately 0.5% of the housing units and 0.5% of the population of
    Allegheny County.
    Market Characteristics for Each Identified Cluster
    Robust Markets
    • “A” markets have the highest housing values, experience the largest level of new construction, have the
    highest owner occupancy levels, and experience little housing distress (such as residential vacancy and
    foreclosure).
    • “B” markets have elevated housing values, experience substantial amounts of new construction, have
    more renters than owners, and experience little housing distress.
    • “C” markets have above average housing values, experience about average levels of new construction,
    have the highest levels of owner occupancy, and experience little housing distress.
    Steady Markets
    • “D” markets have average housing values, experience half the countywide average amount of new
    construction, have more renters than owners, experience average levels of foreclosure, and have low
    levels of vacant lots and poor or worse condition properties.
    • “E” markets have slightly lower than average housing values, experience little new construction, have
    high levels of owner occupancy, have above average amounts of vacant land, and about average levels
    of foreclosure.
    • “F” markets have slightly lower than average housing values, experience slightly above average amounts
    of new construction, have more owners than renters, and have high levels of renters with a subsidy.
    Transitional Markets
    • “G” markets have below average housing values, experience little new construction, have more owners
    than renters, and experience above average levels of foreclosure and residential vacant land.
    • “H” markets have housing values well below the countywide average, experience little new
    construction, have about even numbers of renters and owners, have the highest share of residential
    vacant land, and the highest levels of foreclosure.
    Stressed Markets
    • “I” markets have the second lowest housing values, experience very little new construction, have the
    highest share of renters with a subsidy, experience the highest levels of building violations, and have
    elevated shares of poor or worse condition properties, vacant residential lot area, and foreclosure.
    • “J” markets have the lowest housing values (although there is a substantial amount of variability in
    those prices), more renters than homeowners, the highest share of poor or worse condition properties,
    and elevated shares of building violations, vacant lots, and foreclosure.
    Allegheny County and City of Pittsburgh Market Value Analysis (MVA)
    Average Block Group Housing Market Characteristics - 2021 Allegheny County & City of Pittsburgh MVA
    Table One: Average Block Group Characteristics for 2021 Allegheny County & City of Pittsburgh MVA
    Allegheny County and City of Pittsburgh MVA, 2021
    5
    Executive Summary Map One: Allegheny County and City of Pittsburgh MVA, 2021
    Allegheny County and City of Pittsburgh MVA, 2021
    6
    Executive Summary Map Two: Allegheny County and City of Pittsburgh MVA, 2021
    Allegheny County and City of Pittsburgh MVA, 2021
    7
    Population, Race, and Ethnicity by 2021 MVA Market Type
    Allegheny County and City of Pittsburgh residents most commonly live in Robust market types (about
    44% of County residents), with about 30% of residents in Steady markets, 23% living in Transitional
    markets, and 4% in Stressed markets. Robust and Steady housing markets have prices near or above the
    Countywide average, a mix of highly owner occupied and mixed tenure areas, little housing distress, and
    are where the majority of housing investments are occurring. Black residents are more likely to live in
    Transitional (45%) and Stressed (21%) markets than Robust (11%) or Steady (22%) markets, however.
    Hispanic residents are also more likely to live in Transitional or Stressed markets (32%) than White
    residents (21%). Therefore, People of Color are more likely to live in areas with housing challenges that
    limit their ability to build wealth through home equity and may not have sufficient access to mortgage
    credit.
    Figure Two: Population and Race / Ethnicity by 2021 MVA Market Type
    Changes Since the 2016 MVAs
    Across all market types, sale prices have increased since the 2016 MVAs. In general, market types with
    lower sale prices in 2016 had larger price increases (as a percentage) from 2013-2015 to 2017-2019. The
    Stressed markets experienced the most dramatic proportionate rise in typical sale prices. While that is
    an extraordinary rise, homes in these areas are transacting at prices that are affordable to households at
    approximately one-third the 2015-2019 county median income ($61,043). Sale price increases in all
    market types outstripped inflation. In the City of Pittsburgh, each market type experienced a twenty
    percent or greater sale price increase, including very large increases (both proportionate and raw
    difference) in East End of Pittsburgh, Lawrenceville, and on parts of Northside (‘E’ markets in 2016).
    A 76 120,578 10.1% 8% 2% 2% 86% 0%
    B 113 134,101 11.2% 8% 6% 3% 81% 1%
    C 185 267,610 22.3% 3% 2% 1% 92% 1%
    D 100 108,679 9.1% 7% 10% 3% 76% 4%
    E 196 215,748 18.0% 2% 6% 2%
    output: In Allegheny County, there are various housing markets categorized into different clusters. Let\'s take a closer look at these clusters:

    Robust Markets: These markets have the highest housing values, with a lot of new construction and high levels of owner occupancy. They experience minimal housing distress, such as vacancies and foreclosures.

    Steady Markets: These markets have average housing values, moderate new construction, and a mix of renters and owners. They experience average levels of foreclosure but have low levels of vacant lots and poor-condition properties.

    Transitional Markets: These markets have below-average housing values and little new construction. They have more owners than renters and experience above-average levels of foreclosure and vacant land.

    Stressed Markets: These markets have the lowest housing values, little new construction, and a high share of renters with subsidies. They also have elevated levels of building violations, poor-condition properties, vacant lots, and foreclosures.

    Now, let\'s examine the population distribution across these market types:

    Robust markets are home to about 44% of Allegheny County residents, followed by Steady markets with around 30%.
    Transitional markets house about 23% of the population, while Stressed markets have approximately 4% of the residents.
    Black residents are more likely to live in Transitional (45%) and Stressed (21%) markets compared to Robust (11%) or Steady (22%) markets.
    Hispanic residents also tend to reside in Transitional or Stressed markets (32%) more than White residents (21%).
    These findings suggest that people of color are more likely to face housing challenges, limiting their ability to build wealth through home equity and access mortgage credit.
    Furthermore, there have been changes in sale prices since the 2016 Market Value Analysis (MVA):

    Across all market types, sale prices have increased, with larger percentage increases in market types that had lower prices in 2016.
    Stressed markets experienced the most significant proportionate rise in sale prices, although they still remain affordable for households earning around one-third of the county median income.
    In the City of Pittsburgh, all market types experienced a 20% or greater increase in sale prices, including substantial increases in the East End, Lawrenceville, and parts of the Northside.
    These findings highlight the dynamics of the housing market in Allegheny County and the disparities that exist among different market types. It\'s crucial to address housing challenges in Transitional and Stressed markets, particularly for communities of color, to promote equity and access to homeownership opportunities.

    Summarize the following facts in an engaging format: [input facts]
    {facts}
    output:
    """,
        **parameters
    )
    #print(f"Response from Model: {response.text}")
    return response.text

Genera le domande col language mode

In [5]:
def LMQuiz(context):
    import vertexai
    from vertexai.language_models import TextGenerationModel

    vertexai.init(project="gen-mortadellai", location="us-central1")
    parameters = {
        "temperature": 0.2,
        "max_output_tokens": 256,
        "top_p": 0.8,
        "top_k": 40
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        f"""instructions:
    i want you to generate a series of questions about a given [input_text].
    Generate questions following the following structure example in three rows:
    question:this is an example of a question ?
    options:option1, option2, option3, option4
    answer:the answer to the question**

    [input_text]:
    {context}""",
        **parameters
    )
    print(f"Response from LMQuiz: {response.text}")
    return response.text

Spiega la risposta

In [6]:
def explain(question, context):
    vertexai.init(project="gen-mortadellai", location="us-central1")
    parameters = {
        "temperature": 0.2,
        "max_output_tokens": 256,
        "top_p": 0.8,
        "top_k": 40
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        f"""answer this question and explain the reasoning:
    {question}

    Given this context:
    {context}""",
        **parameters
    )
    print(f"Response from Model: {response.text}")
    return response.text

**Pulizia del testo**

**Suddivisione in chunks**


In [25]:
import unicodedata
import re

def clean_text(input_text):
    text = ""

    for char in input_text:
        try:
            char.encode('utf-8')
            text += char
        except UnicodeEncodeError:
            pass

    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text

def text_to_chunks(text, chunk_size = 10000, overlap_percentage=0.1):
    chunks = []
    overlap_size = int(chunk_size * overlap_percentage)
    start = 0
    end = chunk_size

    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap_size
        end = start + chunk_size
    return chunks

**Sezione relativa al processing del pdf**

per la sintetizzazione riga 43

In [28]:
def extract_text(pdf_file):
    # Open the PDF file
    with open(pdf_file.name, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        # Extract text from each page
        text = ""
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()

    return text

def ask_question(context):
                question = genera_domanda(context)
                question_text = question["question"]
                options = question["options"]
                answer = question["answer"]

                prefix = "question:"
                if question_text.startswith(prefix):
                    question_text = question_text[len(prefix):].strip()

                question_full = question_text + "\n**Possible answers:**\n"
                for option in options:
                    question_full += "\n- " + option

                return question_full, answer


# Interface function
def pdf_to_text(pdf_file, expertise):
    text = extract_text(pdf_file)
    text = clean_text(text)
    chunks = text_to_chunks(text, chunk_size = 3000)
    riassunto = ""
    corpus = ""

    for chunk in chunks:
      riassunto += riassumi(chunk) + '\n'
    intro = romanza(clean_text(riassunto), expertise)
    #sintetizza(intro)
    ultimoPezzo = intro # colpo di tacco
    for chunk in chunks:
      ultimoPezzo = corpo(chunk)
      corpus += ultimoPezzo + '\n'
    podcast = intro + "\n\nLa registrazione si ferma qui\n\n" + corpus + "\n\nFine corpus\n\n"
    return podcast, riassunto

**Sezione relativa al quiz**

Question: What is the pomegranate's nutritional value? Options:

It is a good source of vitamins, minerals, and antioxidants.
It is a good source of fiber.
It is a good source of protein.
It is a good source of carbohydrates. Answer: It is a good source of vitamins, minerals, and antioxidants.

uncomment line 60 for audio generation

In [35]:
import gradio as gr

def format_question(questions):
    if isinstance(questions, dict):
        question_texts = []
        for question_text in questions.values():
            if isinstance(question_text, str):
                question_texts.append(question_text)
    else:
        if isinstance(questions, str):
            question_texts = questions.split("\n\n")
        elif isinstance(questions, list):
            question_texts = [text for text in questions if isinstance(text, str)]
        else:
            return None

    formatted_questions = []
    for question_text in question_texts:
        if question_text.strip() != "":
            question_lines = question_text.strip().split("\n")
            question = {
                "question": question_lines[0].replace("Question: ", ""),
                "options": [option.strip("- ") for option in question_lines[2:-1]],
                "answer": question_lines[-1].replace("Answer: ", "")
            }
            formatted_questions.append(question)

    return formatted_questions

questions = ""
question = ""
import random
def genera_domanda(context):
    global questions
    global question
    if not questions:
        chunks = text_to_chunks(context)
        random_chunk = random.choice(chunks)
        questions = LMQuiz(random_chunk)
        questions = format_question(questions)
        #print(f"New questions generated: {questions}")

    question = questions.pop(0) if questions else None
    #formatted_question = format_question(question) if question else None
    return question

def quiz(answer, podcast):
    domanda = question["question"]
    correct_answer = question["answer"]

    prefix = "answer:"
    if correct_answer.startswith(prefix):
        correct_answer = correct_answer[len(prefix):].strip()

    if answer == correct_answer:
        return "risposta esatta"

    spiegazione = explain(domanda, podcast)
    print("\n\n" + domanda + "\n\n")
    #sintetizza(spiegazione, domanda)
    return "Wrong answer: [" + answer + "] You should have said: " + correct_answer + "\ncontext:\n" + spiegazione

**Audio text to speech**

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

import nltk
nltk.download('punkt')

ci metterà un po'

In [ ]:
preload_models()

In [31]:
def sintetizza(podcast, titolo = "podcast"):
    podcast = podcast.replace("\n", " ").strip()
    sentences = nltk.sent_tokenize(podcast)

    SPEAKER = "v2/en_speaker_6"
    silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

    pieces = []
    for sentence in sentences:
        audio_array = generate_audio(sentence, history_prompt=SPEAKER)
        pieces += [audio_array, silence.copy()]

    concatenated_audio = np.concatenate(pieces)
    audio = Audio(data=concatenated_audio, rate=SAMPLE_RATE)  # Create the Audio object

    audio_array = np.frombuffer(audio.data, dtype=np.int16)  # Convert bytes to NumPy array
    write_wav(titolo, SAMPLE_RATE, audio_array)
    print("\n\noutput.wav salvato in memoria\n\n")

sintetizza in locale

In [ ]:
podcast = "The answer to the question What is Interreg Europe? is that it is a European Union programme that funds interregional cooperation projects. The programme is open to public authorities, public law bodies, and private non-profit bodies from all 27 EU member states, Norway, and Switzerland. Projects must be co-financed by the partners, and the co-financing rate is 80% for public bodies and bodies governed by public law from all 27 EU member states and 70% for private non-profit bodies from all 27 EU member states."

podcast = podcast.replace("\n", " ").strip()
sentences = nltk.sent_tokenize(podcast)

SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]

concatenated_audio = np.concatenate(pieces)
audio = Audio(data=concatenated_audio, rate=SAMPLE_RATE)  # Create the Audio object

audio_array = np.frombuffer(audio.data, dtype=np.int16)  # Convert bytes to NumPy array
write_wav("output.wav", SAMPLE_RATE, audio_array)
print("\n\noutput.wav salvato in memoria\n\n")

**Gradio Web App**

uncomment line 43 for audio generation

In [ ]:
demo = gr.Blocks(theme='freddyaboulton/dracula_revamped')
import time
import random
podcast = ""
riassunto = ""
expertise = ""

with demo:
    gr.Markdown("MortadellAI's podcast generator")
    with gr.Tabs():
        with gr.TabItem("Generate a podcast"):

            radio = gr.Radio(
                ["Beginner", "Intermediate", "Expert"], label="Select your level of expertise on the subject"
            )
            def change_textbox(radio):
                global expertise
                expertise = radio

            radio.change(fn=change_textbox, inputs=radio)
            file_input = gr.File()
            upload_button = gr.Button("upload")
            text_output = gr.Textbox()

        with gr.TabItem("Test yourself!"):
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            clear = gr.ClearButton([msg, chatbot])

            def respond(message, chat_history):
                global podcast
                global riassunto
                bot_message = quiz(message, podcast) + "\n"
                question, answer = ask_question(podcast)
                chat_history.append((message, bot_message + question + "\n" + answer))
                return "", chat_history
            msg.submit(respond, [msg, chatbot], [msg, chatbot])

            def start(file_input, chat_history):
                global podcast, riassunto
                podcast, riassunto = pdf_to_text(file_input, expertise)

                #sintetizza(podcast)

                podcast += "\n\nRIASSUNTO:\n" + riassunto
                print("\n\npodcast:\n" + podcast)
                question, answer = ask_question(podcast)
                chat_history.append(("answer by copying the choosen answer", question + "\n" + answer))
                #gr.Audio(value="output.wav", source="upload")
                return podcast, chat_history


    upload_button.click(start, inputs=[file_input, chatbot], outputs=[text_output, chatbot])


demo.queue().launch(share=True, debug=True)